In [4]:
import ipywidgets as widgets
from IPython.display import display, HTML
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import os


site_list = [
    "stackoverflow",
    "serverfault",
    "superuser",
    "askubuntu",
    "webapps",
    "webmasters",
    "math",
    "physics",
    "chemistry",
    "biology",
    "gaming",
    "anime","politics","english", "travel", "judaism","money", "patents","computer-science","workplace","law","sports","history",
    "philosophy", "economics","electrical-engineering","gardening", "woodworking", "rpg", "android", "security", "mechanics",
    "parenting","lifehacks","datascience","project-management","rasberrypi", "hardware-recs", "coffee","iot", "gis","video",
    "music", "spanish", "literature", "nlp","ai"
]

# CSS Styles
styles = """
<style>
    .custom-dropdown {
        width: 300px;
        font-size: 16px;
    }
    .custom-button {
        font-size: 16px;
        padding: 10px;
    }
    .custom-textarea {
        width: 600px;
        height: 100px;
        font-size: 16px;
    }
    .custom-output {
        border: 1px solid #ccc;
        padding: 10px;
        margin-top: 10px;
    }
</style>
"""
# Function to run a Jupyter notebook
def run_notebook(notebook_path, parameters=None):
    with open(notebook_path) as f:
        nb = nbformat.read(f, as_version=4)
    
    ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
    
    if parameters:
        param_cell = nbformat.v4.new_code_cell(parameters)
        nb.cells.insert(0, param_cell)
    
    try:
        ep.preprocess(nb, {'metadata': {'path': './'}})
    except Exception as e:
        return str(e)
    
    return None

# Define the widget for selecting StackExchange site
site_dropdown = widgets.Dropdown(
    options=site_list,
    description='StackExchange Site:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': 'initial'},
)

# Widget for entering the user question
user_question = widgets.Textarea(
    value='',
    placeholder='Type your question here...',
    description='Question:',
    layout=widgets.Layout(width='600px', height='100px'),
    style={'description_width': 'initial'},
)

# Button to start data collection
collect_button = widgets.Button(
    description='Collect Data',
    disabled=False,
    button_style='',
    tooltip='Click to collect data',
    icon='download',
)

# Output widget to display the data and results
output_area = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin_top='10px'))

In [5]:

# Function to handle data collection

def on_collect_data_button_clicked(b):
    with output_area:
        output_area.clear_output()
        display(HTML("<h3>Collecting data, please wait...</h3>"))
        
        # Run APIdataExtraction notebook with the selected site
        site = site_dropdown.value
        param_str = f"site = '{site}'"
        error = run_notebook('APIdataExtraction.ipynb', param_str)
        
        if error:
            display(HTML(f"<h3 style='color:red;'>Error: {error}</h3>"))
        else:
            display(HTML("<h3>Data collection complete. Proceed to data preprocessing.</h3>"))
            error = run_notebook('DataPreprocessing.ipynb')
            if error:
                display(HTML(f"<h3 style='color:red;'>Error: {error}</h3>"))
            else:
                display(HTML("<h3>Data preprocessing complete. Proceed to clustering.</h3>"))
                error = run_notebook('Clustering.ipynb')
                if error:
                    display(HTML(f"<h3 style='color:red;'>Error: {error}</h3>"))
                else:
                    display(HTML("<h3>Clustering complete. You can now explore the data and get recommendations.</h3>"))

# Attach the collect button click event
collect_button.on_click(on_collect_data_button_clicked)

# Widget for displaying recommended answers
recommendation_output = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin_top='10px'))

# Button to get recommendations
recommend_button = widgets.Button(
    description='Get Recommendations',
    button_style='success',
    layout=widgets.Layout(width='200px', padding='10px'),
)

In [6]:

# Function to handle recommendation
def on_recommend_button_clicked(b):
    with recommendation_output:
        recommendation_output.clear_output()
        question = user_question.value
        print(f"Getting recommendations for: {question}")
        
        # Run the final notebook with the user question as parameter
        final_nb = run_notebook('RecommendationModel.ipynb', question)
        
        # Assume the final notebook stores recommendations in a variable `recommendations`
        recommendations_var = [cell for cell in final_nb.cells if cell.cell_type == 'code' and 'recommended_responses' in cell.source][-1]
        exec(recommendations_var.source, globals())
        
        # Display the recommendations
        display(recommendations_var)

# Attach the recommend button click event
recommend_button.on_click(on_recommend_button_clicked)

# Layout the widgets in the dashboard
dashboard = widgets.VBox([
    widgets.HTML(styles),
    widgets.HBox([site_dropdown, collect_button]),
    output_area,
    user_question,
    widgets.HBox([recommend_button]),
    recommendation_output
])

# Display the dashboard
display(dashboard)